In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
results_2019 = pd.read_csv("../data/bac-results-2019.csv", index_col=0)
results_2020 = pd.read_csv("../data/bac-results-2020.csv", index_col=0)
results_2021 = pd.read_csv("../data/bac-results-2021.csv", index_col=0)

results_2019 = results_2019.apply(pd.to_numeric, errors='coerce').fillna(results_2019)
results_2020 = results_2020.apply(pd.to_numeric, errors='coerce').fillna(results_2020)
results_2021 = results_2021.apply(pd.to_numeric, errors='coerce').fillna(results_2021)

results_2019.fillna('', inplace=True)
results_2020.fillna('', inplace=True)
results_2021.fillna('', inplace=True)

C:\Users\cseke\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
ro = 'romanian_written_final'
mo = 'mother_tongue_written_final'
ma = 'mandatory_subject_written_final'
el = 'elective_subject_written_final'

results_2019_2 = results_2019[results_2019[[ro, mo, ma, el]].applymap(lambda x: np.isreal(x) or x == '').all(1)]

results_all = pd.concat([results_2019_2, results_2020, results_2021], ignore_index=True, sort=False)

nr_romanian_only = len(results_all[results_all['mother_tongue'] == ''])
nr_romanian_plus = len(results_all[results_all['mother_tongue'] != ''])
print(nr_romanian_only)
print(nr_romanian_plus)
print(nr_romanian_only + nr_romanian_plus)

396912
20921
417833


In [6]:
ro2 = 'romanian_written'
mo2 = 'mother_tongue_written'
ma2 = 'mandatory_subject_written'
el2 = 'elective_subject_written'
results_2019_3 = results_2019_2[results_2019_2[[ro2, mo2, ma2, el2]].applymap(lambda x: np.isreal(x) or x == '').all(1)]
results_all = pd.concat([results_2019_3, results_2020, results_2021], ignore_index=True, sort=False)

romanian_only_results = results_all[ \
    (results_all['mother_tongue'] == '') & \
    (results_all['romanian_written_final'] >= 1.0)]
romanian_plus_results = results_all[ \
    (results_all['mother_tongue'] != '') & \
    (results_all['romanian_written_final'] >= 1.0)]

romanian_only_results = romanian_only_results.replace('', np.nan)
romanian_plus_results = romanian_plus_results.replace('', np.nan)

romanian_only_results_filtered = romanian_only_results[ \
    (romanian_only_results['mandatory_subject_written_final'] >= 1.0) & \
    (romanian_only_results['elective_subject_written_final'] >= 1.0)].copy()
romanian_plus_results_filtered = romanian_plus_results[ \
    (romanian_plus_results['mandatory_subject_written_final'] >= 1.0) & \
    (romanian_plus_results['elective_subject_written_final'] >= 1.0) & \
    (romanian_plus_results['mother_tongue_written_final'] >= 1.0)].copy()

romanian_only_results_filtered['inc_ro'] = romanian_only_results_filtered[ro] - romanian_only_results_filtered[ro2]
romanian_only_results_filtered['inc_mo'] = romanian_only_results_filtered[mo] - romanian_only_results_filtered[mo2]
romanian_only_results_filtered['inc_ma'] = romanian_only_results_filtered[ma] - romanian_only_results_filtered[ma2]
romanian_only_results_filtered['inc_el'] = romanian_only_results_filtered[el] - romanian_only_results_filtered[el2]
romanian_only_results_filtered.loc[:, 'inc_avg'] = romanian_only_results_filtered.loc[:, ['inc_ro', 'inc_mo', 'inc_ma', 'inc_el']].sum(axis=1)

romanian_plus_results_filtered['inc_ro'] = romanian_plus_results_filtered[ro] - romanian_plus_results_filtered[ro2]
romanian_plus_results_filtered['inc_mo'] = romanian_plus_results_filtered[mo] - romanian_plus_results_filtered[mo2]
romanian_plus_results_filtered['inc_ma'] = romanian_plus_results_filtered[ma] - romanian_plus_results_filtered[ma2]
romanian_plus_results_filtered['inc_el'] = romanian_plus_results_filtered[el] - romanian_plus_results_filtered[el2]
romanian_plus_results_filtered.loc[:, 'inc_avg'] = romanian_plus_results_filtered.loc[:, ['inc_ro', 'inc_mo', 'inc_ma', 'inc_el']].sum(axis=1)


nr_app_only = len(romanian_only_results[romanian_only_results['mandatory_subject_written_appeal'] >= 1.0])
nr_app_only += len(romanian_only_results[romanian_only_results['elective_subject_written_appeal'] >= 1.0])
nr_app_only += len(romanian_only_results[romanian_only_results['romanian_written_appeal'] >= 1.0])

nr_app_plus = len(romanian_plus_results[romanian_plus_results['mandatory_subject_written_appeal'] >= 1.0])
nr_app_plus += len(romanian_plus_results[romanian_plus_results['elective_subject_written_appeal'] >= 1.0])
nr_app_plus += len(romanian_plus_results[romanian_plus_results['romanian_written_appeal'] >= 1.0])
nr_app_plus += len(romanian_plus_results[romanian_plus_results['mother_tongue_written_appeal'] >= 1.0])

print(nr_app_only/len(romanian_only_results))
print(nr_app_plus/len(romanian_plus_results))

print("\nRomanians' appeal: {:0.3f}".format(romanian_only_results_filtered['inc_avg'].sum() / nr_app_only))
print("minorities' appeal: {:0.3f}".format(romanian_plus_results_filtered['inc_avg'].sum() / nr_app_plus))

0.40380446633573197
0.4321763511553708

Romanians' appeal: 0.203
minorities' appeal: 0.152
